# Maryland Crash Reports - Data Analysis

## Extract

> ### Extract from csv file

In [4]:
import pandas as pd
import numpy as np

In [5]:
df_drivers = pd.read_csv('./Crash_Reporting_-_Drivers_Data.csv', dtype=object)

> ### Describing data

In [7]:
df_drivers.head()

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Name,Off-Road Description,Municipality,...,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Latitude,Longitude,Location
0,DM8479000T,210020119,Takoma Park Police Depart,Property Damage Crash,05/27/2021 07:40:00 PM,NaN,NaN,NaN,IN PARKING LOT,NaN,...,NaN,0,No,Yes,2017,HINO,TWK,38.98765667,-76.987545,"(38.98765667, -76.987545)"
1,MCP2970000R,15045937,MONTGOMERY,Property Damage Crash,09/11/2015 01:29:00 PM,NaN,NaN,NaN,Parking Lot: \n2525 Ennalls Ave,NaN,...,South,5,No,No,2012,TOYOTA,SU,39.03991652,-77.05364898,"(39.03991652, -77.05364898)"
2,MCP20160036,180040948,Montgomery County Police,Property Damage Crash,08/17/2018 02:25:00 PM,NaN,NaN,NaN,PARKING LOT OF 16246 FREDERICK RD,NaN,...,West,15,No,No,2015,MAZD,TK,38.743373,-77.54699707,"(38.743373, -77.54699707)"
3,EJ7879003C,230048975,Gaithersburg Police Depar,Injury Crash,08/11/2023 06:00:00 PM,NaN,NaN,NaN,1 N SUMMIT DRIVE,NaN,...,Unknown,15,No,No,2018,RAM,TK,39.14587303,-77.19194047,"(39.14587303, -77.19194047)"
4,MCP2967004Y,230070277,Montgomery County Police,Property Damage Crash,12/06/2023 06:42:00 PM,Maryland (State),CONNECTICUT AVE,BALTIMORE ST,NaN,KENSINGTON,...,South,35,No,No,2017,AUDI,A3,39.02517017,-77.07633333,"(39.02517017, -77.07633333)"


In [8]:
df_drivers.shape

(201441, 39)

In [9]:
df_drivers['Report Number'].nunique()

113505

In [10]:
df_drivers.isnull().sum()

Report Number                         0
Local Case Number                     0
Agency Name                           0
ACRS Report Type                      0
Crash Date/Time                       0
Route Type                        19740
Road Name                         22403
Cross-Street Name                 34491
Off-Road Description             182741
Municipality                     182315
Related Non-Motorist             194918
Collision Type                      585
Weather                           13356
Surface Condition                 23016
Light                              1445
Traffic Control                   28191
Driver Substance Abuse            31320
Non-Motorist Substance Abuse     196064
Person ID                             0
Driver At Fault                       0
Injury Severity                    1934
Circumstance                     162775
Driver Distracted By               2095
Drivers License State             13317
Vehicle ID                            0


Just from a glance, multiple data points are null and many more are also ambiguous values such as *'Unknown'* and *'Other'*. Also, there are inconsistent values in ***'Agency Name'*** such as *MONTGOMERY* and *Montgomery County Police*. There are 201441 rows (representing the number of individuals/individual vehicles involved in a crash), but only 113505 'Report Numbers' (representing the number of actual crashes). 
<br>
<br>
We will have to clean the data by deciding how to deal with null and ambiguous values as well as standardizing values across the columns.

## Transform

> ### Datatypes (datetime and numeric)

In [14]:
# Crash Date/Time
# convert to datetime object from string in format like '09/19/2024 09:45:00 AM'
df_drivers['Crash Date/Time'] = pd.to_datetime(df_drivers['Crash Date/Time'], format='%m/%d/%Y %I:%M:%S %p')
df_drivers['Crash Date/Time'].dtype

dtype('<M8[ns]')

In [15]:
# Vehicle Year
df_drivers['Vehicle Year'] = pd.to_numeric(df_drivers['Vehicle Year'])
df_drivers['Vehicle Year'].value_counts().sort_index()

Vehicle Year
0       4799
1          2
2          1
3          2
4          1
        ... 
7817       1
8008       1
8888       2
9000       1
9999      64
Name: count, Length: 145, dtype: int64

In [16]:
from datetime import datetime

# get current year
current_year = datetime.now().year

# Arbitrary values, any vehicle year less than 1901 or
# greater than the current_year is deemed invalid
df_drivers.loc[df_drivers['Vehicle Year'] < 1901, 'Vehicle Year'] = 0
df_drivers.loc[df_drivers['Vehicle Year'] > current_year, 'Vehicle Year'] = 0

df_drivers['Vehicle Year'].value_counts().sort_index()

Vehicle Year
0       5066
1901       3
1911       1
1930       1
1938       1
        ... 
2021    4130
2022    3185
2023    2304
2024    1374
2025     342
Name: count, Length: 74, dtype: int64

In [17]:
# Latitude
# Longitude
# 'Longitude' and 'Longitude' column to numeric
df_drivers['Latitude'] = pd.to_numeric(df_drivers['Latitude'])
df_drivers['Longitude'] = pd.to_numeric(df_drivers['Longitude'])

> ### Cleaning data (replacing)

Most columns are mostly standardized and only require few adjustments to maintain consistency. In this case, we will standardize case, add or remove substrings to standardize and replace *'Unknown'* with *NaN* value. 

In [20]:
# Route Type
df_drivers['Route Type'] = df_drivers['Route Type'].replace(' Route', '', regex=True)
df_drivers['Route Type'] = df_drivers['Route Type'].replace('Unknown', np.nan)
df_drivers['Route Type'].value_counts(dropna=False).sort_index()

Route Type
Bicycle                   359
County                  65885
Crossover                 200
Government                748
Interstate (State)       3505
Local                     574
Maryland (State)        87048
Municipality            11487
Other Public Roadway     1382
Private                   232
Ramp                     1125
Service Road               49
Spur                      270
US (State)               8817
NaN                     19760
Name: count, dtype: int64

In [21]:
# Municipality
df_drivers['Municipality'] = df_drivers['Municipality'].str.title()
df_drivers['Municipality'].value_counts(dropna=False).sort_index()

Municipality
Brookeville                16
Chevy Chase #3            156
Chevy Chase #4            325
Chevy Chase #5             74
Chevy Chase View           93
Chevy Chase Village        87
Drummond                   12
Friendship Heights        133
Gaithersburg             6437
Garrett Park               53
Glen Echo                  33
Kensington                413
Laytonsville               25
Matins Addition            28
North Chevy Chase          34
Poolesville               108
Rockville                9320
Somerset                   53
Takoma Park              1685
Washington Grove           41
NaN                    182315
Name: count, dtype: int64

In [22]:
# typo with 'Matins Addition'
df_drivers['Municipality'] = df_drivers['Municipality'].replace('Matins Addition', "Martin's Addition")

In [23]:
# Driver at Fault
df_drivers['Driver At Fault'].value_counts(dropna=False)

Driver At Fault
Yes        102071
No          94688
Unknown      4682
Name: count, dtype: int64

In [24]:
# replace 'Unknown' with NaN
df_drivers['Driver At Fault'] = df_drivers['Driver At Fault'].replace('Unknown', np.nan)

In [25]:
# Injury Severity
df_drivers['Injury Severity'] = df_drivers['Injury Severity'].str.title()
df_drivers['Injury Severity'].value_counts(dropna=False)

Injury Severity
No Apparent Injury          163824
Possible Injury              19450
Suspected Minor Injury       14391
NaN                           1934
Suspected Serious Injury      1657
Fatal Injury                   185
Name: count, dtype: int64

In [26]:
# Injury Severity
df_drivers['Vehicle Damage Extent'].value_counts(dropna=False)

Vehicle Damage Extent
DISABLING               62354
FUNCTIONAL              44993
SUPERFICIAL             43877
Disabling               12408
Superficial              7893
DESTROYED                7610
Functional               6649
UNKNOWN                  6620
NO DAMAGE                6244
Vehicle Not at Scene     2047
No Damage                 328
NaN                       316
OTHER                     102
Name: count, dtype: int64

In [27]:
df_drivers['Vehicle Damage Extent'] = df_drivers['Vehicle Damage Extent'].replace('Unknown', np.nan)

> ### Cleaning data

Some columns have more unstandardized or incorrect values. 
<br>
<br>
As mentioned before, ***'Agency Name'*** has inconsistent values. We will standardize the values by dictionary mapping standardized values and updating the old values.

In [30]:
# function to update column values based on manual dictionary mapping
def dictionary_mapping(column, column_mapping, column_dict={}):
    # create dictionary with default values from column
    for value in df_drivers[column].drop_duplicates().sort_values():
        column_dict[value] = value

    # update default dictionary with manual dictionary mapping
    column_dict.update(column_mapping)
    df_drivers[column] = df_drivers[column].apply(lambda x:column_dict[x])

    # show updated values in column
    print(df_drivers[column].value_counts(dropna=False).sort_index())

In [31]:
# Agency Name
df_drivers['Agency Name'] = df_drivers['Agency Name'].str.title()
df_drivers['Agency Name'].value_counts(dropna=False).sort_index()

Agency Name
Gaithersburg                   2222
Gaithersburg Police Depar      7841
Maryland-National Capital      1154
Mcpark                          337
Montgomery                    35331
Montgomery County Police     138376
Rockville                      2628
Rockville Police Departme      9839
Takoma                          720
Takoma Park Police Depart      2993
Name: count, dtype: int64

In [32]:
agency_name_mapping = {
    'Gaithersburg': 'Gaithersburg Police Department',
    'Gaithersburg Police Depar': 'Gaithersburg Police Department',
    'Maryland-National Capital': 'Maryland-National Capital Park Police Department',
    'Mcpark': 'Maryland-National Capital Park Police Department',
    'Montgomery': 'Montgomery County Police Department',
    'Montgomery County Police': 'Montgomery County Police Department',
    'Rockville': 'Rockville Police Department',
    'Rockville Police Departme': 'Rockville Police Department',
    'Takoma': 'Takoma Park Police Department', 
    'Takoma Park Police Depart': 'Takoma Park Police Department'
}

dictionary_mapping('Agency Name', agency_name_mapping)

Agency Name
Gaithersburg Police Department                       10063
Maryland-National Capital Park Police Department      1491
Montgomery County Police Department                 173707
Rockville Police Department                          12467
Takoma Park Police Department                         3713
Name: count, dtype: int64


In [33]:
df_drivers['Agency Name'].value_counts(dropna=False).sort_index()

Agency Name
Gaithersburg Police Department                       10063
Maryland-National Capital Park Police Department      1491
Montgomery County Police Department                 173707
Rockville Police Department                          12467
Takoma Park Police Department                         3713
Name: count, dtype: int64

Now, all values are standardized and have the same name for each agency. We will continue to apply the same function for similar columns. 

In [35]:
# Weather
df_drivers['Weather'] = df_drivers['Weather'].str.title()
df_drivers['Weather'].value_counts(dropna=False).sort_index()

Weather
Blowing Sand, Soil, Dirt                 15
Blowing Snow                            179
Clear                                139649
Cloudy                                19920
Fog, Smog, Smoke                         64
Foggy                                   676
Freezing Rain Or Freezing Drizzle        53
Other                                   370
Rain                                   3160
Raining                               20609
Severe Crosswinds                        24
Severe Winds                            154
Sleet                                   218
Sleet Or Hail                            12
Snow                                   1713
Unknown                                 878
Wintry Mix                              391
NaN                                   13356
Name: count, dtype: int64

In [36]:
weather_mapping = {
    'Blowing Sand, Soil, Dirt': 'Blowing Sand, Soil, Or Dirt',
    'Fog, Smog, Smoke': 'Fog, Smog, Or Smoke',
    'Foggy': 'Fog, Smog, Or Smoke',
    'Raining': 'Rain',
    'Sleet': 'Sleet Or Hail',
    'Unknown': np.nan
}

dictionary_mapping('Weather', weather_mapping)

Weather
Blowing Sand, Soil, Or Dirt              15
Blowing Snow                            179
Clear                                139649
Cloudy                                19920
Fog, Smog, Or Smoke                     740
Freezing Rain Or Freezing Drizzle        53
Other                                   370
Rain                                  23769
Severe Crosswinds                        24
Severe Winds                            154
Sleet Or Hail                           230
Snow                                   1713
Wintry Mix                              391
NaN                                   14234
Name: count, dtype: int64


In [37]:
# Surface Condition
df_drivers['Surface Condition'] = df_drivers['Surface Condition'].str.title()
df_drivers['Surface Condition'].value_counts(dropna=False).sort_index()

Surface Condition
Dry                         142295
Ice                           1058
Ice/Frost                      158
Mud, Dirt, Gravel               53
Oil                             29
Other                          206
Sand                             8
Slush                          280
Snow                          1136
Unknown                        509
Water (Standing, Moving)         8
Water(Standing/Moving)          41
Wet                          32644
NaN                          23016
Name: count, dtype: int64

In [38]:
surface_condition_mapping ={
    'Ice': 'Ice Or Frost',
    'Ice/Frost': 'Ice Or Frost',
    'Mud, Dirt, Gravel': 'Mud, Dirt, Or Gravel',
    'Unknown': np.nan,
    'Water (Standing, Moving)': 'Water',
    'Water(Standing/Moving)': 'Water'
}

dictionary_mapping('Surface Condition', surface_condition_mapping)

Surface Condition
Dry                     142295
Ice Or Frost              1216
Mud, Dirt, Or Gravel        53
Oil                         29
Other                      206
Sand                         8
Slush                      280
Snow                      1136
Water                       49
Wet                      32644
NaN                      23525
Name: count, dtype: int64


In [39]:
# Light
df_drivers['Light'] = df_drivers['Light'].str.title()
df_drivers['Light'].value_counts(dropna=False).sort_index()

Light
Dark - Lighted                6465
Dark - Not Lighted             991
Dark - Unknown Lighting        175
Dark -- Unknown Lighting      1578
Dark Lights On               39551
Dark No Lights                4966
Dawn                          3760
Daylight                    136966
Dusk                          4321
Other                          435
Unknown                        788
NaN                           1445
Name: count, dtype: int64

In [40]:
light_mapping = {
    'Dark - Lighted': 'Dark (Lighted)',
    'Dark - Not Lighted': 'Dark (Not Lighted)',
    'Dark - Unknown Lighting': 'Dark (Unknown Lighting)',
    'Dark -- Unknown Lighting': 'Dark (Unknown Lighting)',
    'Dark Lights On': 'Dark (Lighted)',
    'Dark No Lights': 'Dark (Not Lighted)',
    'Unknown': np.nan
}

dictionary_mapping('Light', light_mapping)

Light
Dark (Lighted)              46016
Dark (Not Lighted)           5957
Dark (Unknown Lighting)      1753
Dawn                         3760
Daylight                   136966
Dusk                         4321
Other                         435
NaN                          2233
Name: count, dtype: int64


In [41]:
# Vehicle Damage Extent
df_drivers['Vehicle Damage Extent'] = df_drivers['Vehicle Damage Extent'].str.title()
df_drivers['Vehicle Damage Extent'].value_counts(dropna=False).sort_index()

Vehicle Damage Extent
Destroyed                7610
Disabling               74762
Functional              51642
No Damage                6572
Other                     102
Superficial             51770
Unknown                  6620
Vehicle Not At Scene     2047
NaN                       316
Name: count, dtype: int64

> ### Cleaning data (manipulating data)

Other columns require more work to standardize and clean. For ***'Driver Substance Abuse'***, the column has values for both alcohol and drug abuse by the driver. We will convert the column to two separate columns for alcohol and drug abuse and convert the columns to a binary enum - either *'1'*, *'0'*, or *NaN*. 

In [44]:
# Driver Substance Abuse
df_drivers['Driver Substance Abuse'] = df_drivers['Driver Substance Abuse'].str.title()
df_drivers['Driver Substance Abuse'].value_counts(dropna=False).sort_index()

Driver Substance Abuse
Alcohol Contributed                                      1435
Alcohol Present                                          4087
Combination Contributed                                    47
Combined Substance Present                                 92
Illegal Drug Contributed                                  102
Illegal Drug Present                                      259
Medication Contributed                                     64
Medication Present                                        117
None Detected                                          122544
Not Suspect Of Alcohol Use, Not Suspect Of Drug Use     25178
Not Suspect Of Alcohol Use, Suspect Of Drug Use            40
Not Suspect Of Alcohol Use, Unknown                        38
Other                                                      59
Suspect Of Alcohol Use, Not Suspect Of Drug Use           696
Suspect Of Alcohol Use, Suspect Of Drug Use                54
Suspect Of Alcohol Use, Unknown                

In [45]:
driver_substance_abuse_dict = {}
for driver_substance_abuse in df_drivers['Driver Substance Abuse'].drop_duplicates().sort_values():
    driver_substance_abuse_dict[driver_substance_abuse] = driver_substance_abuse

In [46]:
driver_substance_abuse_mapping = {
    'Alcohol Contributed': (1,0),
    'Alcohol Present': (1,0),
    'Combination Contributed': (1,1),
    'Combined Substance Present': (1,1),
    'Illegal Drug Contributed': (0,1),
    'Illegal Drug Present': (0,1),
    'Medication Contributed': (0,1),
    'Medication Present': (0,1),
    'None Detected': (0,0),
    'Not Suspect Of Alcohol Use, Not Suspect Of Drug Use': (0,0),
    'Not Suspect Of Alcohol Use, Suspect Of Drug Use': (0,1),
    'Not Suspect Of Alcohol Use, Unknown': (0,np.nan),
    'Other': (np.nan,np.nan),
    'Suspect Of Alcohol Use, Not Suspect Of Drug Use': (1,0),
    'Suspect Of Alcohol Use, Suspect Of Drug Use': (1,1),
    'Suspect Of Alcohol Use, Unknown': (1,0),
    'Unknown': (np.nan,np.nan),
    'Unknown, Not Suspect Of Drug Use': (np.nan,0),
    'Unknown, Suspect Of Drug Use': (np.nan,1),
    'Unknown, Unknown': (np.nan,np.nan),
}

In [47]:
driver_substance_abuse_dict.update(driver_substance_abuse_mapping)

In [48]:
df_drivers['Driver Alcohol Abuse'] = df_drivers['Driver Substance Abuse'].apply(lambda x:driver_substance_abuse_dict[x][0] if isinstance(x, str) else x)
df_drivers['Driver Drug Abuse'] = df_drivers['Driver Substance Abuse'].apply(lambda x:driver_substance_abuse_dict[x][1] if isinstance(x, str) else x)

# drop 'Driver Substance Abuse' column
df_drivers = df_drivers.drop('Driver Substance Abuse', axis=1)
df_drivers.loc[:, ['Driver Alcohol Abuse', 'Driver Drug Abuse']].value_counts()

Driver Alcohol Abuse  Driver Drug Abuse
0.0                   0.0                  147722
1.0                   0.0                    6307
0.0                   1.0                     582
1.0                   1.0                     193
Name: count, dtype: int64

Where the driver is suspect of alcohol abuse, ***'Driver Alcohol Abuse'*** is *1*; when not suspect, *0*. The same applies for ***'Driver Drug Abuse'***, and unknown values for either column are converted to *NaN*.

In [50]:
# Drivers License State
df_drivers['Drivers License State'].value_counts(dropna=False).sort_index()

Drivers License State
AB         7
AK        23
AL        71
AR        23
AS         3
       ...  
WV       523
WY         9
XX      1851
YT         6
NaN    13317
Name: count, Length: 82, dtype: int64

***'Drivers License State'*** contains mostly US states, but also includes Canada and Mexico. We will create two columns for license state and license country. 

In [52]:
license_state_dict = {}
for license_state in df_drivers['Drivers License State'].drop_duplicates().sort_values():
    license_state_dict[license_state] = license_state

In [53]:
license_state_mapping = {
    'AB': ('AB', 'CA'),
    'AK': ('AK', 'US'),
    'AL': ('AL', 'US'),
    'AR': ('AR', 'US'),
    'AS': ('AS', 'US'),
    'AZ': ('AZ', 'US'),
    'BC': ('BC', 'CA'),
    'CA': (np.nan, 'CA'),
    'CA-ON': ('ON', 'CA'),
    'CA-QC': ('QC', 'CA'),
    'CO': ('CO', 'US'),
    'CT': ('CT', 'US'),
    'DC': ('DC', 'US'),
    'DE': ('DE', 'US'),
    'FL': ('FL', 'US'),
    'FM': (np.nan, 'FM'),    # Federal States of Micronesia assumed
    'GA': ('GA', 'US'),
    'GU': ('GU', 'US'),
    'HI': ('HI', 'US'),
    'IA': ('IA', 'US'),
    'ID': ('ID', 'US'),
    'IL': ('IL', 'US'),
    'IN': ('IN', 'US'),
    'IT': ('IT', np.nan),
    'KS': ('KS', 'US'),
    'KY': ('KY', 'US'),
    'LA': ('LA', 'US'),
    'MA': ('MA', 'US'),
    'MB': ('MB', 'CA'),
    'MD': ('MD', 'US'),
    'ME': ('ME', 'US'),
    'MH': ('MH', 'US'),
    'MI': ('MI', 'US'),
    'MN': ('MN', 'US'),
    'MO': ('MO', 'US'),
    'MP': ('MP', 'US'),
    'MS': ('MS', 'US'),
    'MT': ('MT', 'US'),
    'MX-BCN': ('BC', 'MX'),    # Baja California
    'MX-CMX': ('DF', 'MX'),    # Mexico City
    'MX-GRO': ('GE', 'MX'),    # Guerrero
    'MX-MEX': ('EM', 'MX'),   # State of Mexico
    'MX-ROO': ('QI', 'MX'),   # Quintana Roo
    'NB': ('NB', 'CA'),
    'NC': ('NC', 'US'),
    'ND': ('ND', 'US'),
    'NE': ('NE', 'US'),
    'NF': ('NL', 'CA'),
    'NH': ('NH', 'US'),
    'NJ': ('NJ', 'US'),
    'NL': ('NL', 'CA'),
    'NM': ('NM', 'US'),
    'NS': ('NS', 'CA'),
    'NT': ('NT', 'CA'),
    'NV': ('NV', 'US'),
    'NY': ('NY', 'US'),
    'OH': ('OH', 'US'),
    'OK': ('OK', 'US'),
    'ON': ('ON', 'CA'),
    'OR': ('OR', 'US'),
    'PA': ('PA', 'US'),
    'PR': ('PR', 'US'),
    'QC': ('QC', 'CA'),
    'RI': ('RI', 'US'),
    'SC': ('SC', 'US'),
    'SD': ('SD', 'US'),
    'SK': ('SK', 'CA'),
    'TN': ('TN', 'US'),
    'TX': ('TX', 'US'),
    'UM': ('UM', 'US'),
    'US': (np.nan, 'US'),
    'UT': ('UT', 'US'),
    'VA': ('VA', 'US'),
    'VI': ('VI', 'US'),
    'VT': ('VT', 'US'),
    'WA': ('WA', 'US'),
    'WI': ('WI', 'US'),
    'WV': ('WV', 'US'),
    'WY': ('WY', 'US'),
    'XX': (np.nan, np.nan),
    'YT': ('YT', 'CA')
}

In [54]:
license_state_dict.update(license_state_mapping)

In [55]:
df_drivers['Drivers License Country'] = df_drivers['Drivers License State'].apply(lambda x: license_state_dict[x][1] if isinstance(x, str) else x)
df_drivers['Drivers License State'] = df_drivers['Drivers License State'].apply(lambda x: license_state_dict[x][0] if isinstance(x, str) else x)

We will separate the ***'Vehicle Body Type'*** column into two separate columns for better understanding and convenienece:  ***'Vehicle Class'*** and ***'Vehicle Body Type'***. For example, *'(Sport) Utility Vehicle'* and *'Sport Utility Vehicle'* has *' Sport Utility Vehicle (SUV)'* body type and *'Passenger'* class. 

In [57]:
# Vehicle Body Type 
df_drivers['Vehicle Body Type'] = df_drivers['Vehicle Body Type'].str.title()
df_drivers['Vehicle Body Type'].value_counts(dropna=False).sort_index()

Vehicle Body Type
(Sport) Utility Vehicle                                       15990
All Terrain Vehicle (Atv)                                       119
All-Terrain Vehicle/All-Terrain Cycle (Atv/Atc)                  34
Ambulance/Emergency                                             466
Ambulance/Non Emergency                                         185
Autocycle                                                        45
Bus - Cross Country                                               3
Bus - Mini                                                       16
Bus - Other Type                                                 25
Bus - School                                                    669
Bus - Transit                                                   727
Cargo Van/Light Truck 2 Axles (Over 10,000Lbs (4,536 Kg))      1858
Construction Equipment (Backhoe, Bulldozer, Etc.)                18
Cross Country Bus                                                31
Farm Equipment (Tractor, Combi

In [58]:
vehicle_body_type_mapping = {
    '(Sport) Utility Vehicle': ('Sport Utility Vehicle (SUV)', 'Passenger'),
    'All Terrain Vehicle (Atv)': ('All-Terrain Vehicle/All-Terrain Cycle (ATV/ATC)', 'Off-Road'),
    'All-Terrain Vehicle/All-Terrain Cycle (Atv/Atc)': ('All-Terrain Vehicle/All-Terrain Cycle (ATV/ATC)', 'Off-Road'),
    'Ambulance/Emergency': ('Ambulance', 'Emergency'),
    'Ambulance/Non Emergency': ('Ambulance (Non-Emergency)', 'Emergency'),
    'Autocycle': ('Autocycle', 'Passenger'),
    'Bus - Cross Country': ('Cross Country Bus', 'Bus'),
    'Bus - Mini': ('Minibus', 'Bus'),
    'Bus - Other Type': ('Other', 'Bus'),
    'Bus - School': ('School Bus', 'Bus'),
    'Bus - Transit': ('Transit Bus', 'Bus'),
    'Cargo Van': ('Cargo Van/Light Truck 2 Axles (Over 10,000Lbs (4,536 Kg))', 'Truck'),
    'Cargo Van/Light Truck 2 Axles (Over 10,000Lbs (4,536 Kg))': ('Cargo Van/Light Truck 2 Axles (Over 10,000Lbs (4,536 Kg))', 'Truck'),
    'Construction Equipment (Backhoe, Bulldozer, Etc.)': ('Construction Equipment (Backhoe, Bulldozer, Etc.)', 'Off-Road'),
    'Cross Country Bus': ('Cross Country Bus', 'Bus'),
    'Farm Equipment (Tractor, Combine Harvester, Etc.)': ('Farm Equipment (Tractor, Combine Harvester, Etc.)', 'Off-Road'),
    'Farm Vehicle': ('Farm Vehicle', 'Off-Road'),
    'Fire Vehicle/Emergency': ('Fire Vehicle', 'Emergency'),
    'Fire Vehicle/Non Emergency': ('Fire Vehicle (Non-Emergency)', 'Emergency'),
    'Golf Cart': ('Golf Cart', 'Recreational'),
    'Limousine': ('Limousine', 'Passenger'),
    'Low Speed Vehicle': ('Low Speed Vehicle (LSV)', 'Passenger'),
    'Medium/Heavy Trucks 3 Axles (Over 10,000Lbs (4,536Kg))': ('Medium/Heavy Trucks 3 Axles (Over 10,000Lbs (4,536Kg))', 'Truck'),
    'Moped': ('Moped Or Motorized Bicycle', 'Passenger'),
    'Moped Or Motorized Bicycle': ('Moped Or Motorized Bicycle', 'Passenger'),
    'Motorcycle': ('Motorcycle', 'Passenger'),
    'Motorcycle - 2 Wheeled': ('Motorcycle', 'Passenger'),
    'Motorcycle - 3 Wheeled': ('Motorcycle (3 Wheel)', 'Passenger'),
    'Other': (np.nan, np.nan),
    'Other Bus': ('Other', 'Bus'),
    'Other Light Trucks (10,000Lbs (4,536Kg) Or Less)': ('Other Light Trucks (10,000Lbs (4,536Kg) Or Less)', 'Truck'),
    'Other Trucks': ('Other', 'Truck'),
    'Passenger Car': ('Passenger Car', 'Passenger'),
    'Pickup': ('Pickup Truck', 'Passenger'),
    'Pickup Truck': ('Pickup Truck', 'Passenger'),
    'Police Vehicle/Emergency': ('Police Vehicle', 'Emergency'),
    'Police Vehicle/Non Emergency': ('Police Vehicle (Non-Emergency)', 'Emergency'),
    'Recreational Off-Highway Vehicles (Rov)': ('Recreational Off-Highway Vehicles (ROV)', 'Recreational'),
    'Recreational Vehicle': ('Recreational Vehicle (RV)', 'Recreational'),
    'School Bus': ('School Bus', 'Bus'),
    'Single-Unit Truck': ('Single-Unit Truck', 'Truck'),
    'Snowmobile': ('Snowmobile', 'Recreational'),
    'Sport Utility Vehicle': ('Sport Utility Vehicle (SUV)', 'Passenger'),
    'Station Wagon': ('Station Wagon', 'Passenger'),
    'Transit Bus': ('Transit Bus', 'Bus'),
    'Truck Tractor': ('Truck Tractor', 'Truck'),
    'Unknown': (np.nan, np.nan),
    'Van': ('Van', 'Passenger'),
    'Van - Cargo': ('Cargo Van', 'Truck'),
    'Van - Passenger (&Lt;9 Seats)': ('Passenger Van (Less Than 9 Seats)', 'Passenger'),
    'Van - Passenger (15 Seats)': ('Passenger Van (15 Seats)', 'Passenger'),
    'Van - Passenger (9 Or 12 Seats)': ('Passenger Van (9 Or 12 Seats)', 'Passenger')
}

In [59]:
df_drivers['Vehicle Class'] = df_drivers['Vehicle Body Type'].apply(lambda x: vehicle_body_type_mapping[x][1] if isinstance(x, str) else x)
df_drivers['Vehicle Body Type'] = df_drivers['Vehicle Body Type'].apply(lambda x: vehicle_body_type_mapping[x][0] if isinstance(x, str) else x)

In [60]:
df_drivers.loc[:, ['Vehicle Class','Vehicle Body Type']].value_counts().sort_index()

Vehicle Class  Vehicle Body Type                                        
Bus            Cross Country Bus                                                34
               Minibus                                                          16
               Other                                                           443
               School Bus                                                     3644
               Transit Bus                                                    4369
Emergency      Ambulance                                                       466
               Ambulance (Non-Emergency)                                       185
               Fire Vehicle                                                    437
               Fire Vehicle (Non-Emergency)                                    296
               Police Vehicle                                                 1497
               Police Vehicle (Non-Emergency)                                 2119
Off-Road      

In [61]:
# Circumstance
df_drivers['Circumstance'] = df_drivers['Circumstance'].str.title()
df_drivers[~df_drivers['Circumstance'].isnull()]['Circumstance'].head(20)

12                                           N/A, Wet
16                           Icy Or Snow-Covered, N/A
25                                           N/A, Wet
29           N/A, Road Under Construction/Maintenance
33                                           N/A, Wet
39              Backup Due To Regular Congestion, N/A
43                                    Rain, Snow, Wet
53                                           N/A, Wet
56     N/A, Vision Obstruction (Incl. Blinded By Sun)
67                                    Rain, Snow, Wet
69                                           N/A, Wet
76                                    Rain, Snow, Wet
91                                    Rain, Snow, Wet
98                     Backup Due To Prior Crash, N/A
107                                       Animal, N/A
108                                          N/A, Wet
122                                          N/A, Wet
133                                   Rain, Snow, Wet
134                     Slee

For ***'Circumstance'***, each row can contain multiple circumstances, which are separated by commas. However, some values also contain commas, such as *'Sleet, Hail, Freez. Rain'*. Since the column allows multiple choice, we will first split ***'Circumstance'*** by commas to get a general idea of each value.

In [63]:
pd.set_option('display.max.rows', None)
pd.set_option('display.max_colwidth', None)
# Get all valid responses for 'Circumstance' by first splitting by commas
df_drivers['Circumstance'].drop_duplicates().str.split(', ', expand=True).stack().drop_duplicates().sort_values()

107     0                                                     Animal
571     0                       Backup Due To Non-Recurring Incident
98      0                                  Backup Due To Prior Crash
39      0                           Backup Due To Regular Congestion
20191   0                                               Blowing Sand
470     3                                                      Bumps
172065  3                                                   Careless
357     0                                      Debris Or Obstruction
20191   2                                                       Dirt
172617  0                               Disregard Officer Directions
172335  0                            Disregarded Other Road Markings
172463  0                             Disregarded Other Traffic Sign
172282  6                                                        Etc
172055  0                              Failed To Keep In Proper Lane
173076  0                         

Cross-checking each value and confirming which values contain commas, we can type a manual list with all valid circumstances.

In [65]:
pd.reset_option('display.max.rows')
pd.reset_option('display.max_colwidth')
# all valid circumstances 
circumstance_list = [
    'Animal',
    'Backup Due To Non-Recurring Incident',
    'Backup Due To Prior Crash',
    'Backup Due To Regular Congestion',
    'Blowing Sand, Soil, Dirt',
    'Debris Or Obstruction',
    'Disregard Officer Directions',
    'Disregarded Other Road Markings',
    'Disregarded Other Traffic Sign',
    'Failed To Keep In Proper Lane',
    'Failed To Stop For School Bus',
    'Failed To Yield Right-Of-Way',
    'Followed Too Closely',
    'Icy Or Snow-Covered',
    'Improper Backing',
    'Improper Passing',
    'Improper Turn',
    'N/A',
    'Non-Highway Work',
    'Operated Motor Vehicle In Inattentive, Careless, Negligent, Or Erratic Manner',
    'Operated Motor Vehicle In Reckless Or Aggressive Manner',
    'Other Improper Action',
    'Over-Correcting/Over-Steering',
    'Physical Obstruction(S)',
    'Ran Off Roadway',
    'Ran Red Light',
    'Ran Stop Sign',
    'Road Under Construction/Maintenance',
    'Ruts, Holes, Bumps',
    'Severe Crosswinds',
    'Shoulders Low, Soft, High',
    'Sleet, Hail, Freez. Rain',
    'Smog, Smoke',
    'Snow',
    'Swerved Or Avoided Due To Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist In Roadway, Etc',
    'Toll Booth/Plaza Related',
    'Too Fast For Conditions',
    'Traffic Control Device Inoperative',
    'V Exhaust System|R Other Road',
    'V Wipers|W Other Environmental',
    'Vision Obstruction (Incl. Blinded By Sun)',
    'Wet',
    'Worn, Travel-Polished Surface',
    'Wrong Side',
    'Wrong Way',
    'Rain'    #placed at end to prevent interefering with another value containing 'Freez. Rain'
]

In [66]:
# convert circumstance list to dictionary for mapping
circumstance_dict = {}
circumstance_dict = {circumstance: circumstance for circumstance in circumstance_list}

In [67]:
circumstance_mapping = {
    'Blowing Sand, Soil, Dirt': 'Blowing Sand, Soil, Dirt',
    'Disregard Officer Directions': 'Disregarded Officer Directions',
    'N/A': '',
    'Operated Motor Vehicle In Inattentive, Careless, Negligent, Or Erratic Manner': 'Inattentive, Careless, Negligent Or Erratic Manner',
    'Operated Motor Vehicle In Reckless Or Aggressive Manner': 'Reckless Or Aggressive Manner',
    'Over-Correcting/Over-Steering': 'Over-Correcting Or Over-Steering',
    'Physical Obstruction(S)': 'Physical Obstructions',
    'Road Under Construction/Maintenance': 'Road Under Construction Or Maintenance',
    'Ruts, Holes, Bumps': 'Ruts, Holes, Or Bumps',
    'Shoulders Low, Soft, High': 'Shoulders Low, Soft, Or High',
    'Sleet, Hail, Freez. Rain': 'Sleet, Hail, Or Freezing Rain',
    'Smog, Smoke': 'Smog Or Smoke',
    'Swerved Or Avoided Due To Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist In Roadway, Etc': 'Swerved Or Avoided Due To Hazards',
    'Toll Booth/Plaza Related': 'Toll Booth Or Plaza Related',
    'Too Fast For Conditions': 'Too Fast For Conditions',
    'V Exhaust System|R Other Road': 'Vehicle Exhaust System Or Other Road',
    'V Wipers|W Other Environmental': 'Vehicle Wipers With Other Environmental Factors',
    'Vision Obstruction (Incl. Blinded By Sun)': 'Vision Obstruction (Sun Incl.)',
    'Worn, Travel-Polished Surface': 'Worn Or Travel-Polished Surface',
}

In [68]:
circumstance_dict.update(circumstance_mapping)

In [69]:
# following code block will first replace old values in column with values in circumstance_dict,
# then explode each value into individual rows, essentially converting from wide to long format.
import re

# replace according to circumstance_mapping for more concise and consistent wording 
pattern = re.compile('|'.join(re.escape(circumstance) for circumstance in circumstance_dict.keys()))
df_drivers['Circumstance'] = df_drivers['Circumstance'].apply(lambda x: pattern.sub(lambda m: circumstance_dict[m.group(0)], x) if isinstance(x, str) else x)

# separate all rows by values in circumstance_list, then explode into individual rows
pattern = r'|'.join(re.escape(circumstance) for circumstance in sorted(circumstance_dict.values(), key=len, reverse=True))
regex = re.compile(pattern)
df_drivers['Circumstance'] = df_drivers['Circumstance'].apply(lambda x: regex.findall(x) if isinstance(x, str) else x)
df_drivers = df_drivers.explode('Circumstance').reset_index(drop=True)
df_drivers['Circumstance'] = df_drivers['Circumstance'].replace('', np.nan)    # all empty rows converted to np.nan after exploded
df_drivers['Circumstance'].drop_duplicates().sort_values()

165                                                  Animal
972                    Backup Due To Non-Recurring Incident
153                               Backup Due To Prior Crash
54                         Backup Due To Regular Congestion
37940                              Blowing Sand, Soil, Dirt
605                                   Debris Or Obstruction
330568                       Disregarded Officer Directions
330035                      Disregarded Other Road Markings
330258                       Disregarded Other Traffic Sign
329611                        Failed To Keep In Proper Lane
331307                        Failed To Stop For School Bus
329630                         Failed To Yield Right-Of-Way
329621                                 Followed Too Closely
19                                      Icy Or Snow-Covered
330674                                     Improper Backing
329660                                     Improper Passing
329633                                  

## Load

In [71]:
df_drivers.dtypes

Report Number                            object
Local Case Number                        object
Agency Name                              object
ACRS Report Type                         object
Crash Date/Time                  datetime64[ns]
Route Type                               object
Road Name                                object
Cross-Street Name                        object
Off-Road Description                     object
Municipality                             object
Related Non-Motorist                     object
Collision Type                           object
Weather                                  object
Surface Condition                        object
Light                                    object
Traffic Control                          object
Non-Motorist Substance Abuse             object
Person ID                                object
Driver At Fault                          object
Injury Severity                          object
Circumstance                            

In [72]:
print('Number of Reports:', df_drivers['Report Number'].nunique())
print('Number of Individuals:', df_drivers['Person ID'].nunique())
print('Total Number of Circumstances:', df_drivers.shape[0])

Number of Reports: 113505
Number of Individuals: 201441
Total Number of Circumstances: 376061


***'Person ID'*** is unique for each individual and ***'Report Number'*** is unique for each crash. Since the dataframe has been blown up for each value in ***'Circumstance'***, the total amount of rows now represent the total number of each individual circumstance.
<br>
<br>
We will create a new dataframe with interesting data for data analysis and export to csv files. One dataframe will include rows with all circumstances exploded while the other will only include one row for each individual. 

In [74]:
df_export = df_drivers[[
    'Report Number', 'ACRS Report Type', 'Crash Date/Time', 'Route Type', 
    'Municipality', 'Weather', 'Surface Condition', 'Light', 'Person ID', 
    'Injury Severity', 'Circumstance', 'Drivers License State', 'Drivers License Country',
    'Vehicle Body Type', 'Vehicle Class', 'Latitude', 'Longitude'
]]

In [136]:
df_export.to_csv('data/MarylandCrashReports.csv', index=False)

In [76]:
# drop circumstance column
df_no_circumstance = df_drivers[[
    'Report Number', 'ACRS Report Type', 'Crash Date/Time', 'Route Type', 
    'Municipality', 'Weather', 'Surface Condition', 'Light', 'Person ID', 
    'Injury Severity', 'Drivers License State', 'Drivers License Country',
    'Vehicle Body Type', 'Vehicle Class', 'Latitude', 'Longitude'
]]

In [77]:
# drop duplicates from exploding 'Circumstance'
df_no_circumstance = df_no_circumstance.drop_duplicates()

In [138]:
df_export.to_csv('data/MarylandCrashReports_NoCircumstance.csv', index=False)